<a href="https://colab.research.google.com/github/tazkera-haque-ds/tazkera-haque-ds.github.io/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

In [ ]:
# Install dependencies (run once)
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py
!wget https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/utils.py
!pip install -r requirements.txt

--2023-11-08 20:24:52--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      82  --.-KB/s    in 0s      

2023-11-08 20:24:52 (867 KB/s) - ‘requirements.txt’ saved [82/82]

--2023-11-08 20:24:52--  https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 295 [text/plain]
Saving to: ‘param

# EDA
<!--- @wandbcode{course-lesson1} -->

In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data.

In [ ]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [ ]:
from fastai.vision.all import *
import params

import wandb

We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually.

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets.

In [ ]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [ ]:
path = Path(untar_data(URL, force_download=True))

In [ ]:
path.ls()

Here we define several functions to help us process the data and upload it as a `Table` to W&B.

In [ ]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)

    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )

    return table

We will start a new W&B `run` and put everything into a raw Artifact.

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

In [ ]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

Let's add the images and label masks.

In [ ]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table.

In [ ]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [ ]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`.

In [ ]:
raw_data_at.add(table, "eda_table")

In [ ]:
run.log_artifact(raw_data_at)
run.finish()